## Variations in Hospital Charge and Determinants in USA
### Monopolistic Competition in Healthcare Sector

Generally speaking, USA healthcare sector behaves as monopolistic competition that allows the providers to set their own prices for same or similar services. The monopolistic power to the providers comes from either real or perceived differences in quality of medical and related services. In health sector new technologies can also be used to signal quality even when their clinical usefulness is not proven.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from pandas import ExcelWriter
from pandas import ExcelFile
import geopandas as gpd
import json
import folium
from collections import Counter
import seaborn as sns
rcParams['figure.figsize'] = 10.5, 8.5

## About the Data

####  - 49 States and DC, excluding Maryland
####  - 35 Procedures(Ambulatory Payment Classifications)
####  -The data represents 15.9% (428 million) of Outpatient Prospective Payment System(OPPS) hospital services and  23.3% ($14.3        billion) of the total  Medicare allowed amount charges.
####  - Only One year Data is used because of differences in data structure for previous years.
####  - All inferences and conclusions are based on and for 2016 Only.

In [ ]:
# This is hospital information data, is not charge data.
# This data is obtained from CMS website
Hospital_Information=pd.read_csv('data/Hospital_General_Information.csv')
Hospital_Information.head()

In [ ]:
Counter(Hospital_Information['Patient experience national comparison'])

In [ ]:
Hospital_Information.columns

In [ ]:
# Keep only columns we want to use
Hospital_Information=Hospital_Information[['Provider ID', 'Hospital Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name','Hospital Type', 'Hospital Ownership','Patient experience national comparison',
       'Emergency Services','Effectiveness of care national comparison','Efficient use of medical imaging national comparison','Readmission national comparison',
       'Hospital overall rating','Location']]

In [ ]:
# Import 2016 Medicare Outpatient Hospital charge data 

OutPatmedicarecharge=pd.read_csv('data/Provider_Outpatient_Hospital_Charge_Data_by_APC__CY2016.csv')
OutPatmedicarecharge.head()
#OutPatmedicarecharge=pd.read_csv('data/Provider_Outpatient_Hospital_Charge_Data_by_APC__CY2015.csv')
#OutPatmedicarecharge.head()
# to join charge data with hospital information data, we take upper cases of string columns 

OutPatmedicarecharge['Provider Name']=OutPatmedicarecharge['Provider Name'].apply(lambda x: x.upper())
OutPatmedicarecharge['Provider Street Address']=OutPatmedicarecharge['Provider Street Address'].apply(lambda x: x.upper())
OutPatmedicarecharge['Provider City']=OutPatmedicarecharge['Provider City'].apply(lambda x: x.upper())
OutPatmedicarecharge['Provider State']=OutPatmedicarecharge['Provider State'].apply(lambda x: x.upper())


### Average Estimated Submitted Charges
The provider's average estimated submitted charge for services covered by Medicare for the APC. The average charges submitted vary from hospital to hospital because of differences in hospital charge structures.
### Average Total Payments
The average of total payments to the provider for the APC including the Medicare Payments for APCs. Average Total Payments also include co-payment and deductible amounts that the patient is responsible for.
### Average Medicare Payment Amount
The amount Medicare Paid to the hospitals or providers.

### Total Observations: the data has total of 27652 APC services at different hospitals participating in OPPS.
### The overall Average charge submitted by hospitals is $41,633.24 (It is average of average charges)

In [ ]:
# Check desctriptiptive statistics
OutPatmedicarecharge.drop(['Provider ID','Provider Zip Code','APC'],axis=1).describe().round(2)

In [ ]:
# check state incuded in the data
OutPatmedicarecharge['Provider State'].unique()

In [ ]:
# create State Dictionary
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [ ]:
# convert from State dictionary to data frame 
state_df=pd.DataFrame.from_dict(us_state_abbrev, orient='index')\
         .reset_index()
# rename columns   
state_df.columns=['State','Provider State']
state_df.head()

In [ ]:
# import zipcodes by state and county data
#zipcodes=pd.read_csv('data/ZIP-COUNTY-FIPS_2017-06.csv')
#print(zipcodes.head())

# create a county column by replace County from COUNTYNAME column
#zipcodes['County']=zipcodes.COUNTYNAME.str.replace('(County)','',regex=True)
# keep only three columns listed
#zipcodes=zipcodes[['ZIP','County','STATE']]

# rename the columns
#zipcodes.columns=['Provider Zip Code','County','Provider State']
# merge zipcode data with state_df
#state_data=pd.merge(state_df,zipcodes, how= 'inner', on='Provider State' )
#state_data.head()

In [ ]:
## merge State data with Outpatient charge data
#merged_df=pd.merge(state_data,OutPatmedicarecharge, how='inner', on=['Provider State','Provider Zip Code'])
#merged_df.head()

### Large gap between Medicare Allowed charges and what the Providers charge

In [ ]:
# plot hospital charges, medicare allowed amount, and medicare payments
OutPatmedicarecharge.iloc[:,10:13].describe().iloc[1,0:4].plot(kind='bar')
plt.title('Outpatient Hospital Charges')
plt.ylabel("amount in US $")

#### We can also see State Level Average State By State for the three types of charges

In [ ]:
OutPatmedicarecharge.head()

In [ ]:
charge_data=pd.merge(state_df,OutPatmedicarecharge,how='inner',on='Provider State')
charge_data.shape

In [ ]:
# create state list
state_list=state_df['State'].unique().tolist()

### This function creates interactive tools

In [ ]:
### This function creates interactive analysis
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def f(x):
    OutPatmedicarecharge_df=charge_data
    OutPatmedicarecharge_df=OutPatmedicarecharge_df[OutPatmedicarecharge_df['State']==x]
    by_state_desc = OutPatmedicarecharge_df.iloc[:,11:14].describe()
    by_state_desc.iloc[1,0:4].plot(kind='bar')
    state_plot=plt.title('Outpatient Hospital Charges')
    plt.ylabel("amount in US $")
    return state_plot

In [ ]:
interactive(f,x=state_list)

In [ ]:
Hospital_Information.columns

In [ ]:
# rename hospital information data to match with charge data(hospital charge data)
Hospital_Information.columns=['Provider ID', 'Provider Name', 'Provider Street Address', 
                              'Provider City', 'Provider State', 'Provider Zip Code',
                              'County','Hospital Type', 'Hospital Ownership',
                              'Patient experience national comparison','Emergency Services',
                              'Effectiveness of care national comparison',
                              'Efficient use of medical imaging national comparison',
                              'Readmission national comparison', 'Hospital overall rating',
                              'Location']

In [ ]:
# merge charge_data with hospital Informaation data
Merged_df2=pd.merge(charge_data,Hospital_Information, how='left',on =['Provider ID', 'Provider Street Address', 'Provider City', 'Provider State', 'Provider Zip Code'])
print('shape :', Merged_df2.shape)
Merged_df2.columns


In [ ]:
# drop  duplicate columns
Merged_df2=Merged_df2.drop('Provider Name_y',axis=1)

In [ ]:
# rename columns
Merged_df2=Merged_df2.rename(columns={'Provider Name_x':'Provider Name'})
Merged_df2.info()

In [ ]:
# extract lat and Long data from parethesis
Merged_df2['lat_Long']=Merged_df2.Location.str.extract('\(([^)]+)\)',expand=True)
# break lat_long column into lat and long columns
Merged_df2[['Lat','Long']]=Merged_df2.lat_Long.str.split(',',expand=True)
Merged_df2.head()

### Majority of the services are provided by  'Voluntary non-profit - Private': total of 13,188

In [ ]:
# Count of hospital by ownership type
Counter(Merged_df2['Hospital Ownership'])

In [ ]:
# count of services taken places by hospitals ownership type
Merged_df2[['Hospital Ownership','Provider Name']]\
.groupby('Hospital Ownership').count().sort_values('Provider Name',ascending=False)

In [ ]:
# count of Voluntary non-profit - Private Hospitals
len(Merged_df2[Merged_df2['Hospital Ownership']=='Voluntary non-profit - Private']['Provider Name'].unique())

## Ten unique Federal Government Hospitals data are incuded

In [ ]:
# Check list of Goverment Hospitals
Counter(Merged_df2[Merged_df2['Hospital Ownership']=='Government - Federal']['Provider Name'])

### Hospital charge by ownership type shows proprietary hospitals charged the highest 
### Local Government hospitals charged lowest amount
### Tribal is the lowest but there is one Tribal provider in the data.

In [ ]:
# plot hospital charges by Ownership Type

Merged_df2[['Hospital Ownership','Average Estimated Total Submitted Charges']]\
.groupby('Hospital Ownership').median().sort_values('Average Estimated Total Submitted Charges',ascending=False)\
.plot(kind='bar',color='firebrick')

plt.ylabel("Hospital charges in US $")
plt.xlabel("Ownership Type")
plt.title("Median Hospital Charges by Ownership Type")

In [ ]:
Counter(Merged_df2['Patient experience national comparison'])

### Hopsitals rated below national average in terms of Patient experience national comparison have highest average charge
### Hospitals rated above national average has lowest charge compared with both below and same as national average.
### 1162 providers are missing the information, 228 providers Have no Rating Available.

In [ ]:
# plot average submitted charges by rating based on customer expriences
Merged_df2[['Patient experience national comparison','Average Estimated Total Submitted Charges']]\
.groupby('Patient experience national comparison').mean().plot(kind='bar')

In [ ]:
Merged_df2.columns

In [ ]:
# plot average submitted charges by rating based on customer expriences
Merged_df2[['Efficient use of medical imaging national comparison','Average Estimated Total Submitted Charges']]\
.groupby('Efficient use of medical imaging national comparison').mean().plot(kind='bar')
plt.ylabel("in USD")
plt.xlabel("Efficient use of medical imaging national comparison")
plt.title('Hospital Charges by Efficieny of Hospital in Using Medical Imaging')


In [ ]:
Merged_df2.isna().sum()

In [ ]:
### Drop the columns with most missing values 

Merged_df2=Merged_df2.drop(['Outlier Comprehensive APC Services','Average Medicare Outlier Amount'],axis=1)            

In [ ]:
Merged_df2.head()

In [ ]:
# Split Ownership into Goverment, Voluntary-non-profit and Proprietary
#Merged_df2['Hospital Ownership'].str.contains('Voluntary',regex=True)
#Merged_df2[['Ownership_type','ignore1','ignore2']]=Merged_df2['Hospital Ownership'].str.split('-',expand=True)
#Merged_df2.columns

In [ ]:
#Merged_df2=Merged_df2.drop(['ignore1','ignore2'],axis=1)

In [ ]:
Merged_df2[['Government - Federal', 'Government - Hospital District or Authority',
       'Government - Local', 'Government - State', 'Physician', 'Proprietary',
       'Tribal', 'Voluntary non-profit - Church',
       'Voluntary non-profit - Other', 'Voluntary non-profit - Private']]=pd.get_dummies(Merged_df2['Hospital Ownership'])

In [ ]:
# create dummy variables for ownership type

#Merged_df2[['Government','Physician','Proprietary','Tribal','Voluntary non-profit']]=pd.get_dummies(Merged_df2.Ownership_type)

In [ ]:
Merged_df2.columns

In [ ]:
# create dummy variables for ownership type
Merged_df2[['Above the national average','Below the national average',
            'Not Available','Same as the national average']]=pd.get_dummies(Merged_df2['Effectiveness of care national comparison'])

Qaulity_ownership_4=Merged_df2[['Hospital Ownership','Above the national average','Below the national average',
                              'Not Available','Same as the national average']].groupby('Hospital Ownership').sum()

Qaulity_ownership_4['Total_Ownership_category']=Qaulity_ownership_4[Qaulity_ownership_4.columns].sum(axis=1)
# Effectiveness of care by Ownership type
Qaulity_in_percent_4=Qaulity_ownership_4.iloc[:,0:-1].div(Qaulity_ownership_4.iloc[:,4], axis=0).round(2)*100
# drop Tribal and display
Qaulity_in_percent_4=Qaulity_in_percent_4.drop('Tribal',axis=0)
Qaulity_in_percent_4['Same or Above National Average']=Qaulity_in_percent_4.iloc[:,0]+Qaulity_in_percent_4.iloc[:,3]
Qaulity_in_percent_4

In [ ]:
# readmission  by types of ownership
Qaulity_in_percent_4[['Above the national average','Below the national average','Same as the national average']].plot(kind='bar')
plt.ylabel("Effectiveness of Care in percentage")
plt.xlabel("Hospital Ownership Type")
plt.title("Effectiveness of Care national comparison By Hospital Ownership Type")

### State Government Hospitals have least readmission rate, followed by Federal Government Hospitals
### Physicians services the highest readmission rate, followed by Voluntary non-profit - Church hospitals

In [ ]:
# create dummy variables for ownership type
Merged_df2[['Above the national average','Below the national average',
            'Not Available','Same as the national average']]=pd.get_dummies(Merged_df2['Readmission national comparison'])

Qaulity_ownership_3=Merged_df2[['Hospital Ownership','Above the national average','Below the national average',
                              'Not Available','Same as the national average']].groupby('Hospital Ownership').sum()

Qaulity_ownership_3['Total_Ownership_category']=Qaulity_ownership_3[Qaulity_ownership_3.columns].sum(axis=1)
# Readmision by Ownership type
Qaulity_in_percent_3=Qaulity_ownership_3.iloc[:,0:-1].div(Qaulity_ownership_3.iloc[:,4], axis=0).round(2)*100
# drop Tribal and display
Qaulity_in_percent_3=Qaulity_in_percent_3.drop('Tribal',axis=0)
Qaulity_in_percent_3['Same or Below National Average']=Qaulity_in_percent_3.iloc[:,1]+Qaulity_in_percent_3.iloc[:,3]
Qaulity_in_percent_3

In [ ]:
# readmission  by types of ownership
Qaulity_in_percent_3[['Above the national average','Below the national average','Same as the national average']].plot(kind='bar')
plt.ylabel("Readmission national comparison in percentage")
plt.xlabel("Hospital Ownership Type")
plt.title("Readmission national comparison By Hospital Ownership Type")

### Propreitary Hospitals Tends to use Medical Imaging More than other categories(Less efficient) 
### Federal Government is least user of Medical Imaging(Most Efficient)

In [ ]:
# create dummy variables for ownership type
Merged_df2[['Above the national average','Below the national average',
            'Not Available','Same as the national average']]=pd.get_dummies(Merged_df2['Efficient use of medical imaging national comparison'])

Qaulity_ownership_2=Merged_df2[['Hospital Ownership','Above the national average','Below the national average',
                              'Not Available','Same as the national average']].groupby('Hospital Ownership').sum()

Qaulity_ownership_2['Total_Ownership_category']=Qaulity_ownership_2[Qaulity_ownership_2.columns].sum(axis=1)
# Medical Imaging rating by Ownership type
Qaulity_in_percent_2=Qaulity_ownership_2.iloc[:,0:-1].div(Qaulity_ownership_2.iloc[:,4], axis=0).round(2)*100
# drop Tribal and display
Qaulity_in_percent_2=Qaulity_in_percent_2.drop('Tribal',axis=0)
Qaulity_in_percent_2

In [ ]:
# Medical Imaging by types of ownership
Qaulity_in_percent_2[['Above the national average','Below the national average','Same as the national average']].plot(kind='bar')
plt.ylabel("Medical Imaging Efficieny in percentage")
plt.xlabel("Hospital Ownership Type")
plt.title("Efficiency of Medical Imaging By Hospital Ownership Type")

In [ ]:
# create dummy variables for ownership type
Merged_df2[['Above the national average','Below the national average',
            'Not Available','Same as the national average']]=pd.get_dummies(Merged_df2['Patient experience national comparison'])

In [ ]:
Qaulity_ownership=Merged_df2[['Hospital Ownership','Above the national average','Below the national average',
                              'Not Available','Same as the national average']].groupby('Hospital Ownership').sum()

### The patient expreince by percentage  

In [ ]:
Qaulity_ownership['Total_Ownership_category']=Qaulity_ownership[Qaulity_ownership.columns].sum(axis=1)
# Patient Expreince rating by Ownership type
Qaulity_in_percent=Qaulity_ownership.iloc[:,0:-1].div(Qaulity_ownership.iloc[:,4], axis=0).round(2)*100
# drop Tribal and display
Qaulity_in_percent=Qaulity_in_percent.drop('Tribal',axis=0)
Qaulity_in_percent

### Highest Level of Satisfactions of Patient with Physicians and State Hospitals 
### This may imply, it would make sense for the Federal government to give more discretion on healthcare investment and administration to States

In [ ]:
# patient Experience by types of ownership
Qaulity_in_percent[['Above the national average','Below the national average','Same as the national average']].plot(kind='bar')
plt.ylabel("Patient Satisfaction in Percentage")
plt.title("Patient Satisfaction By Hospital Ownership Type")

### Implantable Cardioverter Defibrillator (ICD) Procedures is at the top of charges : median charge is about $116372.42
### ICD is used to monitor and control heart in patients that had critical heart attacks 
### The next high charge was for Level6 ENT procedures(Ear ,Nose and Throat surgery)
### Procedures related to comprehensive general obsevations, lower level electrophycological procedures , gynocology and urology are at bottom of charges. 

In [ ]:
# plot median charge by APC Procedures
Merged_df2[['APC Description','Average Estimated Total Submitted Charges']]\
.groupby('APC Description').median().sort_values('Average Estimated Total Submitted Charges',ascending=False)\
.plot(kind='bar')

###  map Hospital charge and Informations

### - map all hospitals against overall median charges for outpatient charges in 2016
       used choropleth for this.
### - map for ICD procedures map using against ICD median charges
### - map for comprehensive obseervation services

In [ ]:
Merged_df2[['Average Estimated Total Submitted Charges']].median()

In [ ]:
Merged_df2.columns

In [ ]:
Counter(Merged_df2['APC Description'])

In [ ]:
# filter Comprehensive Observation Services Procedures from the data
Merged_df3=Merged_df2[Merged_df2['APC Description']=='Comprehensive Observation Services']
# keep only columns wanted be popped up in the map and group by Provider Name and County, Lat  and Long and calculate median
Merged_df3=Merged_df3[['State','Provider HRR','Hospital Ownership','Average Estimated Total Submitted Charges','Lat','Long']]\
        .groupby(['State','Provider HRR','Hospital Ownership','Lat','Long']).median()\
        .reset_index()

In [ ]:
# find overall median charge for ICD and Similar Procedures
median_chargeCompObs=Merged_df2[Merged_df2['APC Description']=='Comprehensive Observation Services']\
                                                              ['Average Estimated Total Submitted Charges'].median()
median_chargeCompObs=pd.to_numeric(median_chargeCompObs)
median_chargeCompObs=round(median_chargeCompObs,2)

In [ ]:
115525.388519813

In [ ]:
# Filter only Implantable Cardioverter Defibrillator (ICD) Procedures
# It is used to monitor and control heart in patients that had critical heart attacks 
Merged_df4=Merged_df2[Merged_df2['APC Description']=='Level 2 ICD and Similar Procedures']

# keep only columns wanted be popped up in the map and group by Provider Name and County, Lat  and Long and calculate median
Merged_df4=Merged_df4[['State','Provider HRR','Hospital Ownership','Average Estimated Total Submitted Charges','Lat','Long']]\
        .groupby(['State','Provider HRR','Hospital Ownership','Lat','Long']).median()\
        .reset_index()

In [ ]:
# find overall median charge for ICD and Similar Procedures
median_chargeICD=Merged_df2[Merged_df2['APC Description']=='Level 2 ICD and Similar Procedures']['Average Estimated Total Submitted Charges'].median()
median_chargeICD=pd.to_numeric(median_chargeICD)
round(median_chargeICD,2)

In [ ]:
Merged_df4.head()

In [ ]:
### reassing Merged_df3 to mapCompObs_df and drop missing, and convert Lat and Long to numeric
mapCompObs_df=Merged_df3
mapCompObs_df=mapCompObs_df.dropna(axis=0)
mapCompObs_df['Lat']=pd.to_numeric(mapCompObs_df.Lat)
mapCompObs_df['Long']=pd.to_numeric(mapCompObs_df.Long)

In [ ]:
### reassing Merged_df4 to Merged_df_map and drop missing, and convert Lat and Long to numeric
Merged_df_map=Merged_df4
Merged_df_map=Merged_df_map.dropna(axis=0)
Merged_df_map['Lat']=pd.to_numeric(Merged_df_map.Lat)
Merged_df_map['Long']=pd.to_numeric(Merged_df_map.Long)
Merged_df_map['Average Estimated Total Submitted Charges']= Merged_df_map['Average Estimated Total Submitted Charges'].round(2)
Merged_df_map.head()

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='asidamo', api_key='mKLD0mhVoFbiAx3Z0qZW')

In [ ]:
state_loc=pd.read_csv('data/statelatlong.csv')

state_loc.shape

In [ ]:
state_loc.head()

In [ ]:
state_loc.columns=['State_abbr','Lat,','Long','State']

In [ ]:
#group the data by State
State_median= Merged_df2[['State','Average Estimated Total Submitted Charges']].groupby(['State']).mean()

In [ ]:
Median_State_charge=pd.merge(State_median,state_loc,how='left',on='State')
Median_State_charge.head()

In [ ]:
df = Median_State_charge ## pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['State'] 

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['State_abbr'],
        z = df['Average Estimated Total Submitted Charges'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = " Average Hospital Charge in USD"
        )
    ) ]

layout = dict(
        title = 'Average Hospital Charge for Medicare Outpatient in 2016',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

fig = dict(data=data, layout=layout)

url = py.plot(fig, filename='d3-cloropleth-map')

In [ ]:
   # Full derates to map
map_df = Merged_df_map# Merged_df_map.State=='Tennessee']
    #create our map of the world and show it
map_world = folium.Map(location = [36.1627,-86.78], zoom_start = 5)

# draw our plate boundaries: plate_bounds
# folium.GeoJson(plate_bounds).add_to(map_world)

#iterate through map_df to create locations and markers for each piece

for row in map_df.iterrows():
    row_values = row[1] 
    location = [row_values['Lat'], row_values['Long']]
    popup ='Referral Region: ' + str(row_values['Provider HRR']) + '<br/>' + 'Ownership: ' + str(row_values['Hospital Ownership']) + '<br/>'+ 'APC Procedure: ' + 'Level2 ICD and Similar Procedures' + '<br/>'+'Median Hospital Charge: ' + str(row_values['Average Estimated Total Submitted Charges'])

#marker = folium.Marker(location = location, popup = popup)
#         marker = folium.Circle(location=location, popup=popup, radius=mag_radius, color='crimson', fill=False, fill_color='crimson').add_to(map_world)
#         marker.add_to(map_world)

    charge = row_values['Average Estimated Total Submitted Charges']
    if charge >150000:
        color = 'red'
    elif charge > median_chargeICD:# median hospital charge for Implantable Cardioverter Defibrillator (ICD) is $116525

        color = 'yellow'
    else:
        color = 'green'
    marker = folium.Circle(location=location, popup=popup, radius=20000, color=color, fill=False, fill_color=color)# .add_to(map_world)
    marker.add_to(map_world)

# Show map
map_world

In [ ]:
#Merged_df_map[~Merged_df_map.Lat.str.contains("...") == True]

## Below functions helps us to map state by state

In [ ]:
# create state list
state_list=Merged_df_map['State'].unique().tolist()

In [ ]:
def f(x):
    
    # Full derates to map
    map_df = Merged_df_map[Merged_df_map.State==x]
  
    #create our map of the world and show it
    map_world = folium.Map(location = [36.1627,-86.78], zoom_start = 5)

    # draw our plate boundaries: plate_bounds
    # folium.GeoJson(plate_bounds).add_to(map_world)
        
    for row in map_df.iterrows():
        row_values = row[1] 
        location = [row_values['Lat'], row_values['Long']]
        popup ='Referral Region: ' + str(row_values['Provider HRR']) + '<br/>' + 'Ownership: ' + str(row_values['Hospital Ownership']) + '<br/>'+ 'APC Procedure: ' + 'Level2 ICD and Similar Procedures' + '<br/>'+'Median Hospital Charge: ' + str(row_values['Average Estimated Total Submitted Charges'])
    
    #for row in map_df.iterrows():
    #    row_values = row[1] 
    #    location = [row_values['Lat'], row_values['Long']]
    #    popup ='State: ' + x + '<br/>' + 'APC Procedure: ' + 'Level 2 ICD and Similar Procedures' + '<br/>' +  'Median Hospital Charge: ' + str(row_values['Average Estimated Total Submitted Charges'])
             
        #marker = folium.Marker(location = location, popup = popup)
    
    #         marker = folium.Circle(location=location, popup=popup, radius=mag_radius, color='crimson', fill=False, fill_color='crimson').add_to(map_world)
    #         marker.add_to(map_world)
   
        charge = row_values['Average Estimated Total Submitted Charges']
        if charge >150000:
            color = 'red'
        elif charge > median_chargeICD:# median hospital charge for Implantable Cardioverter Defibrillator (ICD) is $115525.39
            color = 'yellow'
        else:
            color = 'green'
        marker = folium.Circle(location=location, popup=popup, radius=20000, color=color, fill=False, fill_color=color)# .add_to(map_world)
        marker.add_to(map_world)

   
    return map_world

In [ ]:
# this part interacts and filters only one State

interact(f, x=state_list);

In [ ]:
procedures=Merged_df2['APC Description'].unique().tolist()

# this part interacts and filters only one State

interact(f, x=state_list,y=procedures);

In [ ]:
# replace level  and level numbers
# replace text level with blank
Merged_df2['ProcedureName']=Merged_df2['APC Description'].str.replace('Level','')

# create a new column called level, to control the complexity of treatments
Merged_df2['Level'] = Merged_df2['APC Description'].str.extract('(\d)', expand=True)


In [ ]:
# replace numbers with blank
Merged_df2['ProcedureName']=Merged_df2['ProcedureName'].replace('\d+','',regex=True)

# check spaces and then we will trim them 

Merged_df2['ProcedureName'].unique()

In [ ]:
# remove space at beginning from colum values using str.strip
Merged_df2['ProcedureName']=Merged_df2['ProcedureName'].str.strip()
Merged_df2['ProcedureName'].unique()

In [ ]:
Merged_df2.info()

In [ ]:
# convert level to nmeric data type
Merged_df2['Level']=pd.to_numeric(Merged_df2.Level)

In [ ]:
Merged_df2.Level.dtype

### Nevada is at top in terms of  median hospital charge: median hospital charge $48532.31

### Followed by Florida ,median charge ($46254.79)

In [ ]:
# calculate median average charges by States
Merged_df2[['State','Average Estimated Total Submitted Charges']]\
.groupby('State').median().sort_values('Average Estimated Total Submitted Charges',ascending=False).round(2).head(5)

In [ ]:
# median Average charge by State
Merged_df2[['State','Average Estimated Total Submitted Charges']]\
.groupby('State').median().sort_values('Average Estimated Total Submitted Charges',ascending=False).round(2).plot(kind='bar')
plt.ylabel('Average Estimated Tot Charge in US $')
plt.title('Median Average Estimated Tot Charges By States')

In [ ]:
#plot number of services provided vs average hospital charge by provider
Merged_df2.plot(x='Comprehensive APC Services',y='Average Estimated Total Submitted Charges',kind="scatter")
plt.xlabel('# of Comprehensive APC Services')

In [ ]:
# remove space at beginning from colum values using regex
#merged_df['ProcedureName']=merged_df['ProcedureName'].replace('^\s+','',regex=True)
#merged_df['ProcedureName'].unique()

In [ ]:
# remove space at end from colum values
#merged_df['ProcedureName']=merged_df['ProcedureName'].replace('\s+$','',regex=True)
#merged_df['ProcedureName'].unique()

In [ ]:
# calculate the difference between the average estimated total sumbitted charges and 
#merged_df['submitted_allowed_diff']=(merged_df['Average Estimated Total Submitted Charges']-merged_df['Average Medicare Allowed Amount'])

In [ ]:
#merged_df['submitted_allowed_diff'].sum()


In [ ]:
# create Tennessee and FLorida
#OutPatmedicarecharge_TN=merged_df[merged_df['Provider State'].str.contains('TN')]
#OutPatmedicarecharge_FL=merged_df[merged_df['Provider State'].str.contains('FL')]

# distribution
#OutPatmedicarecharge_heartProc=OutPatmedicarecharge_TN[OutPatmedicarecharge_TN['APC Description']=='Level 3 Endovascular Procedures']
#OutPatmedicarecharge_heartProc.iloc[:,12:15].plot(kind='box')
#plt.title('Distribution of Outpatient Charges by Hospitals for Level 3 Endovascular Procedures in Tennessee')
#plt.ylabel("amount in US $")


In [ ]:
# show distribution of charges by each APC procedures

def f(x):
    OutPatmedicarecharge_heartProc=Merged_df2[Merged_df2['APC Description']==x]
    OutPatmedicarecharge_heartProc.iloc[:,11:14].plot(kind='box')
    plot_proc=plt.title('Distribution of Outpatient Charges by Hospitals for Level 3 Endovascular Procedures in USA')
    plt.ylabel("amount in US $")
    
    return plot_proc

In [ ]:
interact(f, x=procedures);

In [ ]:
# show distribution of charges by each APC procedures

def f(x,y):
    OutPatmedicarecharge_heartProc=Merged_df2[(Merged_df2['State']==x) &(Merged_df2['APC Description']==y)]
    OutPatmedicarecharge_heartProc.iloc[:,11:14].plot(kind='box')
    plot_proc_by_state=plt.title('Distribution of Outpatient Charges by State and Procedure Type')
    plt.ylabel("amount in US $")
    
    return plot_proc_by_state

In [ ]:
interact(f,x=state_list, y=procedures);

In [ ]:
# plot by Providers Name
def f(x):
    Merged_df_Provider=Merged_df2[Merged_df2['State']==x].reset_index()
    Merged_df_Provider=Merged_df_Provider[['State','Provider Name','Average Estimated Total Submitted Charges']].groupby(['Provider Name','State']).median().sort_values('Average Estimated Total Submitted Charges',ascending=False)
    Merged_df_Provider=Merged_df_Provider.head(20).reset_index()
    plot_provider=plt.bar(x=Merged_df_Provider['Provider Name'],height= Merged_df_Provider['Average Estimated Total Submitted Charges'])
    plt.xticks(rotation='vertical')

    return plot_provider

In [ ]:
interact(f,x=state_list);

In [ ]:
Merged_df2.columns

# plot charges by Providers and Procedures
def f(x):
    Merged_df_Provider=Merged_df2[Merged_df2['State']==x]
    Merged_df_Provider_1=Merged_df_Provider[['State','ProcedureName','Average Estimated Total Submitted Charges']]\
                         .groupby(['State','ProcedureName']).median().sort_values('Average Estimated Total Submitted Charges',ascending=False)\
                         .reset_index()
            
    plot_provider=plt.bar(x=Merged_df_Provider_1['ProcedureName'],height= Merged_df_Provider_1['Average Estimated Total Submitted Charges'])
    plt.xticks(rotation='vertical')
    
    return plot_provider
    

interact(f,x=state_list);

In [ ]:
# plot charges by category of procedurs 
OutPatmedcharge_Proc=Merged_df2[['Provider State','ProcedureName','Average Estimated Total Submitted Charges',
       'Average Medicare Allowed Amount', 'Average Medicare Payment Amount']]
OutPatmedcharge_byProcedure=OutPatmedcharge_Proc.groupby('ProcedureName').median().sort_values('Average Estimated Total Submitted Charges', ascending=False)
OutPatmedcharge_byProcedure.head(10)

OutPatmedcharge_byProcedure.plot(kind='bar')
plt.title('Charges by Type of Procedures')
plt.xlabel('Procedure Type')
plt.ylabel('amount in US $')

### In Most States the charge is distributed per complexity of the procedures
### comperhensive observation lies at the bottom and ICD lies at the top

In [ ]:
def f(x):
    # plot charges by category of procedurs 
    OutPatmedcharge_Proc=Merged_df2[['State','ProcedureName','Average Estimated Total Submitted Charges',
           'Average Medicare Allowed Amount', 'Average Medicare Payment Amount']]
    # filter state
    OutPatmedcharge_Proc=OutPatmedcharge_Proc[OutPatmedcharge_Proc.State==x]
    # sort ascending order by charge amount
    OutPatmedcharge_byProcedure=OutPatmedcharge_Proc.groupby('ProcedureName').median()\
                               .sort_values('Average Estimated Total Submitted Charges', ascending=False)
   

    plot_proc=OutPatmedcharge_byProcedure.plot(kind='bar')
    plt.title('Charges by Type of Procedures')
    plt.xlabel('Procedure Type')
    plt.ylabel('amount in US $')

    return plot_proc


In [ ]:
interact(f,x=state_list);

In [ ]:
OutPatmedicarecharge['APC Description'].unique()

In [ ]:
# Median differences in submitted and allowed charges by states
#OutPatmedicarecharge[['Provider State' ,'submitted_allowed_diff']].groupby('Provider State').median().sort_values('submitted_allowed_diff',ascending=False)

In [ ]:
# total differences in submitted and allowed charges by APC categories
#(OutPatmedicarecharge[['APC Description' ,'submitted_allowed_diff']].groupby('APC Description').sum()/1000000).sort_values('submitted_allowed_diff',ascending=False)

In [ ]:
#OutPatmedicarecharge[['Provider City' ,'submitted_allowed_diff']].groupby('Provider City').sum().sort_values('submitted_allowed_diff',ascending=False)

In [ ]:
# inpatPatmedicarecharge=pd.read_csv('data/State_Summary_of_Inpatient_Charge_Data_by_Medicare_Severity_Diagnosis_Related_Group__MS-DRG___FY2015.csv')

In [ ]:
#inpatPatmedicarecharge.info()

In [ ]:
### compare mean variation in medicare payments and hospital charges by State 
### compare mean variation in medicare payments and hospital charges by State and DRG

## import county health ranking data 

In [ ]:
# import county health ranking data 

countyhealthrank_df=pd.read_excel('data/2016 County Health Rankings Data - v3.xls',sheet_name='Additional Measure Data', index_col=0,skiprows=1)

In [ ]:
countyhealthrank_df.columns

### my variables= ['State','County','Age-Adjusted Mortality','% 65 and over','Child Mortality Rate','% African American', '% Non-Hispanic White','Household Income','Other PCP Rate','% Uninsured','# Motor Vehicle Deaths','# Drug Overdose Deaths','Drug Overdose Mortality Rate','MV Mortality Rate','% Limited Access','% Diabetic','% Frequent Mental Distress']


In [ ]:
# keep only needed columns
healthranking_df=countyhealthrank_df[['State','County','Age-Adjusted Mortality','% 65 and over',
                                      'Child Mortality Rate','% African American', '% Non-Hispanic White',
                                      'Household Income','Other PCP Rate','% Uninsured adults','% Uninsured children',
                                      '# Motor Vehicle Deaths','# Drug Overdose Deaths','Drug Overdose Mortality Rate',
                                      'MV Mortality Rate','% Limited Access to Healthy Foods','% Diabetic',
                                      '% Frequent Mental Distress','% Rural']].reset_index()

In [ ]:
healthranking_df.columns

In [ ]:
healthranking_df.head()

In [ ]:
# capitalize County to match with hospital charge data, str(x) will convert the State column to string.
# without doing that in this particular data I got this error "AttributeError: 'float' object has no attribute 'upper"
healthranking_df['County']=healthranking_df.County.apply(lambda x: str(x).upper())

In [ ]:
# group by State and County 
hospitalCharge=Merged_df2[['State','County','Comprehensive APC Services','Average Estimated Total Submitted Charges']]\
                   .groupby(['State','County']).mean().reset_index()

In [ ]:
hospitalCharge.head()

In [ ]:
# strip  county column
hospitalCharge['County']=hospitalCharge.County.str.strip()

In [ ]:
combined_df=pd.merge(hospitalCharge,healthranking_df,how='left',on=['State','County'])
combined_df.head()

In [ ]:
combined_df.shape

In [ ]:
# plot relationship between rural  % of communities and hospital charges

In [ ]:
plt.scatter(combined_df['% Rural'],combined_df['Average Estimated Total Submitted Charges'])
plt.title('Relationship between Hospital Charges and Percentage of Rural Communities in a County')
plt.ylabel('hospital charges in US $')
plt.xlabel('percentage of Rural communities')

In [ ]:
plt.scatter(combined_df['% 65 and over'],combined_df['Average Estimated Total Submitted Charges'])
plt.title('Relationship between Hospital Charges and Percentage of Over 65 Population')
plt.ylabel('hospital charges in US $')
plt.xlabel('percentage of Over 65 Population')

In [ ]:
plt.scatter(combined_df['Other PCP Rate'],combined_df['Average Estimated Total Submitted Charges'])
plt.title('Relationship between Hospital Charges and Other PCP Rate')
plt.ylabel('hospital charges in US $')
plt.xlabel('Other PCP Rate')

In [ ]:
from sklearn.cluster import KMeans
X=combined_df[['Average Estimated Total Submitted Charges']]
Kmean = KMeans(n_clusters=4)
Kmean.fit(X)

In [ ]:
Kmean.cluster_centers_

In [ ]:
plt.scatter(combined_df['% Uninsured adults'],combined_df['Average Estimated Total Submitted Charges'])
plt.title('Relationship between Hospital Charges and Percentage of Uninsured adults')
plt.ylabel('hospital charges in US $')
plt.xlabel('percentage of Uninsured adults')

In [ ]:
plt.scatter(combined_df['% Limited Access to Healthy Foods'],combined_df['Average Estimated Total Submitted Charges'])
plt.title('Relationship between Hospital Charges and Percentage of Limited Access to Healthy Foods')
plt.ylabel('hospital charges in US $')
plt.xlabel('% of Population With Limited Access to Healthy Foods')

In [ ]:
#salary=pd.read_excel("data/state_M2016_dl.xlsx")

#salary.head()

In [ ]:
#salary=salary[salary.OCC_TITLE=='Physicians and Surgeons, All Other']
#salary=salary[['STATE','OCC_TITLE','A_MEAN']]
#salary.columns=['State','OCC_TITLE','AveYearlySalary']
#salary.sort_values('AveYearlySalary',ascending=False).plot(kind='bar',x='State',y='AveYearlySalary')

In [ ]:
# merge salary data
#combined_data=pd.merge(combined_df,salary, how='left',on='State')
#combined_data.columns

In [ ]:
# import linear regression model
import scipy.stats as stats
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing  # helps for reshaping X and y
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import stats


### Use Lassso Regression it will filters out less important variables

In [ ]:
combined_df=combined_df.dropna(axis=0)

In [ ]:
# shape of the data after dropping na horizontally 
combined_df.shape

In [ ]:
# convert Other PCP rate to numeric
combined_df['Other PCP Rate']=pd.to_numeric(combined_df['Other PCP Rate'])
# print data info
combined_df.info()

In [ ]:
# drop FIPS
combined_df=combined_df.drop(['FIPS'],axis=1)

In [ ]:
combined_df['Comprehensive APC Services']=combined_df['Comprehensive APC Services']

In [ ]:
combined_df.columns

In [ ]:
# re-order the data frame so that hospital charge comes on 3rd column
combined_df=combined_df[['State', 'County',
       'Average Estimated Total Submitted Charges','Comprehensive APC Services',
       '% 65 and over', '% African American',
       '% Non-Hispanic White', 'Household Income', 'Other PCP Rate',
       '% Uninsured adults', '% Uninsured children', '# Motor Vehicle Deaths',
       '# Drug Overdose Deaths', 'Drug Overdose Mortality Rate','% Limited Access to Healthy Foods', '% Diabetic','% Rural','Age-Adjusted Mortality','Child Mortality Rate','MV Mortality Rate','% Frequent Mental Distress']]

In [ ]:
combined_df.head()

In [ ]:
combined_df.shape

In [ ]:
# plot correlation between variables
combined_corr_df=combined_df.iloc[:,2:21]
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(combined_corr_df.corr(),cmap='coolwarm',annot=True);

In [ ]:
# drop mortality rates and '% Frequent Mental Distress data because they have strong correlation with other variables
combined_df=combined_df.drop(['Age-Adjusted Mortality','Child Mortality Rate','MV Mortality Rate','% Frequent Mental Distress'],axis=1)


combined_corr_df=combined_df.iloc[:,2:21]
corr = combined_corr_df.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm',vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(combined_corr_df.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(combined_corr_df.columns)
ax.set_yticklabels(combined_corr_df.columns)
plt.show()


In [ ]:
# plot correlation between variables after dropping strongly correlated variables
combined_corr_df=combined_df.iloc[:,2:21]
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(combined_corr_df.corr(),cmap='coolwarm',annot=False);

In [ ]:
# define X and y variables
y=combined_df.iloc[:,2]
#X=combined_df.iloc[:,3:21]
# 
#X=combined_df.iloc[:,-1]abs
X=combined_df[['% Uninsured adults','% Rural','% 65 and over']]


In [ ]:
y.head()

In [ ]:
X.head()

In [ ]:
# reshape it becuase it is only one feature
#X=X.values.reshape(-1,1)

#X=combined_df[['% 65 and over','Child Mortality Rate',
       '% African American','Household Income',
       'Other PCP Rate', '% Uninsured adults', '% Uninsured children',
       '# Motor Vehicle Deaths', '# Drug Overdose Deaths',
       'Drug Overdose Mortality Rate', 'MV Mortality Rate',
       '% Limited Access to Healthy Foods', '% Diabetic',
       '% Frequent Mental Distress']]

In [ ]:
# split the data into train and test, 33% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# define lasso model

clf = linear_model.Lasso(alpha=0.1) 
# fit and predict with lasso model
clf.fit(X_train,y_train)
train_score=clf.score(X_train,y_train)
test_score=clf.score(X_test,y_test)
coeff_used = np.sum(clf.coef_!=0)

# print scores
print("training score:", train_score) 
print("test score: ", test_score)
print("number of features used: ", coeff_used)

print(clf.coef_)

In [ ]:
# define the model
lmodel=LinearRegression()
lmodel.fit(X_train,y_train)
# predict the model
y_pred=lmodel.predict(X_test)


In [ ]:
print('R score: %.2f' % r2_score(y_test, y_pred))
print ('Mean Square Error:%.2f' % mean_squared_error(y_test,y_pred))

In [ ]:
lmodel.coef_

### Counties with more  % of Uninsured population are positively  related with hospital charges 
 #### i.e. Hospitals are shifting costs
### 1 % increase in % of uninsured increases  charge increses hospital charges by $642.0305


### 1% increase in proportion of Rural communities in the counties decreases hospital charges by $433.7657

In [ ]:
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
# tried  OLS model  selected variables by county level

# define y
# define X and y variables
y=combined_df.iloc[:,2]

X=  combined_df[['% 65 and over','% Non-Hispanic White', 'Household Income',
       '% Uninsured adults', '% Uninsured children', '# Motor Vehicle Deaths','# Drug Overdose Deaths', '% Limited Access to Healthy Foods', '% Diabetic','Other PCP Rate','% Rural']]

In [ ]:
X.shape

In [ ]:
# import lasso
from sklearn.linear_model import Lasso

In [ ]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Lasso model penalizes large coeffient variables, and removes irrelevant variables by making the coefficient zero.
### However, in this case the model didn't droop any variable

In [ ]:

clf = linear_model.Lasso(alpha=1, normalize=True) 
# fit and predict with lasso model
clf.fit(X_train,y_train)
train_score=clf.score(X_train,y_train)
test_score=clf.score(X_test,y_test)
coeff_used = np.sum(clf.coef_!=0)

# print scores
print("training score:", train_score) 
print("test score: ", test_score)
print("number of features used: ", coeff_used)

print(clf.coef_)



#### % of % of elrderly population, % of uninsured(both adults and children),
#### % of diabetic population, have positive relation with hospitals charge

#### Household Income, Motor Vehicle Deaths, Other to Primary Care Physucains ratio  have very small positive relation with hospital charges.

In [ ]:
# plot lasso coeficients
lasso_coef=clf.fit(X,y).coef_
plt.plot(range(len(X.columns)),lasso_coef)
plt.xticks(range(len(X.columns)),X.columns,rotation=60)
plt.show()

In [ ]:
# define the model
lmodel_st=LinearRegression()
lmodel_st.fit(X_train,y_train)
# predict the model
y_pred=lmodel_st.predict(X_test)
print('R score: %.2f' % r2_score(y_test, y_pred))
print ('Mean Square Error:%.2f' % mean_squared_error(y_test,y_pred))
lmodel_st.coef_

### - Limited access to healthy food is highly associated particularly with  those living in low-income neighborhoods,communities of color, and rural areas
### - This could be a possible explanation why it has negative effect on hospital charges. 
### - Low R-squared score  and very large constant term imply that hospital charge may not be determined by competition in the market rather it is influenced by the hospitals charge master which is mainly administrative decision. 

In [ ]:
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
combined_df[combined_df['% Rural']>60.00].shape

In [ ]:
# group the data by state
combined_State_df=combined_df.drop('County',axis=1).groupby('State').mean()
combined_State_df.head()


In [ ]:
# save file to csv
#state_median_charge.to_csv('data/state_median_charge.csv',index=False)

### Import 2014 and 2015 data

In [ ]:
outpatientcharge2014=pd.read_csv('data/Outpatient_Prospective_Payment_System__OPPS__Provider_Summary_for_32_Selected_Ambulatory_Payment_Classification__APC__Groups_-_CY2014.csv')
outpatientcharge2014.head()

In [ ]:
outpatientcharge2015=pd.read_csv('data/Provider_Outpatient_Hospital_Charge_Data_by_APC__CY2015.csv')
outpatientcharge2015.head()

In [ ]:
# 2015 State level median hospital charges
state_median_charge2015=outpatientcharge2015[['Provider State','Average Estimated Total Submitted Charges']].groupby('Provider State').median()
# reset index
state_median_charge2015=state_median_charge2015.reset_index()

In [ ]:
state_median_charge2015=pd.merge(state_df,state_median_charge2015, how='inner', on='Provider State')
state_median_charge2015.head()

In [ ]:
stateMedianCharge2015=state_median_charge2015.drop('Provider State',axis=1)

In [ ]:
stateMedianCharge2015.to_csv('data/state_median_charge2015.csv',index=False)

In [ ]:
outpatientcharge2013=pd.read_csv('data/Medicare_Provider_Charge_Outpatient_APC30_CY2013_v2.csv')
outpatientcharge2013.head()

In [ ]:
oupatientcharge2012=pd.read_csv('data/Medicare_Provider_Charge_Outpatient_APC30_CY2012.csv')

oupatientcharge2012.head()

In [ ]:
print(oupatientcharge2012['Average  Estimated Submitted Charges'].mean())
print(Merged_df2['Average Estimated Total Submitted Charges'].mean())

In [ ]:
#county_location=gpd.read_file('data/County_Location.geojson')
#county_location.head()

In [ ]:
#county_location=county_location[['COUNTY_NAME','geometry']]\
#.rename(columns={'COUNTY_NAME':'County'})
#county_location.head()

In [ ]:
#combined_df['County']=combined_df['County'].apply(lambda x: x.upper())

# Things to do on Today

- Use Quality Indicators data from hospital information  data
- Check national comparison for readmission 
- Do Every Analysis for one Procedure: Example Level 3 Endovascular Procedure.